In [1]:
import sys
if "../src" not in sys.path:
    sys.path.append("../src")

In [2]:
import tarfile
import gzip

from tqdm.auto import tqdm
from itertools import islice
from datetime import datetime
import textwrap

import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
output_paths = []

## Data Dependencies

```
../data/di-morphosyntax.csv 0f1c87 (LF)
```

In [3]:
from hashlib import sha1
from pathlib import Path
paths = ["../data/di-morphosyntax.csv"]
for path_x in paths:
    h = sha1()
    h.update(Path(path_x).read_bytes())
    print(path_x, h.hexdigest()[:6])

../data/di-morphosyntax.csv 0f1c87


## Load dependencies

In [4]:
data = pd.read_csv("../data/di-morphosyntax.csv", index_col=0)
pd.value_counts(data.MorphoSyntax), pd.value_counts(data.MorphoSyntax).sum()
mr_data = data.dropna(axis=0, subset=["MorphoSyntax"])
print(mr_data.shape)
mr_data.head()

(1676, 8)


,token,source,ASBC,Apple (2006-2016),China (2015-2016),Dcard (2019-2020),PTT (2004-2019),MorphoSyntax
4073,丟來,Corpus,1.0,5.0,5.0,1.0,0.0,VR
4074,丟光,Corpus,2.0,16.0,6.0,1.0,0.0,VR
4077,丟出,Corpus,9.0,190.0,56.0,16.0,0.0,VR
4079,丟到,Corpus,53.0,265.0,75.0,48.0,0.0,VR
4087,丟命,"MOE, Corpus",1.0,5.0,2.0,0.0,0.0,VO


In [5]:
mr_data.MorphoSyntax.value_counts()

VR    879
AV    340
VV    244
VO    213
Name: MorphoSyntax, dtype: int64

In [6]:
mr_data_path = "../data/verb_morpho.csv"
output_paths.append(mr_data_path)
mr_data.to_csv(mr_data_path, line_terminator="\n")

In [7]:
from itertools import chain
chars = set(list(chain(*mr_data.token)))
words = mr_data.token.tolist() + list(chars)
print("nChar, n(bisyll-)Word: ", len(chars), len(mr_data.token))
print("nVocab: ", len(words))

nChar, n(bisyll-)Word:  885 1676
nVocab:  2561


In [8]:
gzf = gzip.open("../data/tencent-ailab-embedding-zh-d200-v0.2.0-s.tar.gz", "rb")
tarf = tarfile.open(fileobj=gzf)

word_set = set(words)
tarinfo = tarf.getmember('tencent-ailab-embedding-zh-d200-v0.2.0-s/tencent-ailab-embedding-zh-d200-v0.2.0-s.txt')
embf = tarf.extractfile(tarinfo)
nvocab, ndim = [int(x) for x in 
                embf.readline().decode().strip().split()]
ln = embf.readline().decode().strip()
emb_vecs = []
vocabs = []

pbar = tqdm(total=nvocab)
while ln:
    pbar.update(1)
    toks = ln.split()    
    tok0 = toks[0]
    
    if tok0 in word_set:
        vec = [float(x) for x in toks[1:]]
        vocabs.append(tok0)
        emb_vecs.append(vec)
    ln = embf.readline().decode().strip()
pbar.close()

  0%|          | 0/2000000 [00:00<?, ?it/s]

In [9]:
kv = KeyedVectors(ndim)

In [10]:
kv.add_vectors(vocabs, emb_vecs)

In [11]:
kv.most_similar("住")

[('回', 0.562965452671051),
 ('近', 0.5430368185043335),
 ('租', 0.5377331376075745),
 ('搬去', 0.5235447287559509),
 ('破', 0.5212136507034302),
 ('定', 0.5209269523620605),
 ('起', 0.520687460899353),
 ('呆', 0.5123663544654846),
 ('到', 0.5034065842628479),
 ('去', 0.5023194551467896)]

In [12]:
kv.most_similar("泛出")

[('淌下', 0.7242736220359802),
 ('淌出', 0.7215964198112488),
 ('映成', 0.6647475361824036),
 ('激射', 0.6409773230552673),
 ('映照', 0.6285261511802673),
 ('迸', 0.6251155138015747),
 ('仰起', 0.6060652732849121),
 ('充盈', 0.5890071392059326),
 ('吞吐', 0.5870674848556519),
 ('逼出', 0.5773579478263855)]

In [13]:
emb_paths = "../data/verb_morpho_emb.txt"
kv.save_word2vec_format(emb_paths)
output_paths.append(emb_paths)

## Words found in Tencent

In [14]:
word_list = vocabs
word_list = sorted(word_list, key=lambda x: (len(x),x))
print("Not found: ", len(word_list))
print(*textwrap.wrap(" ".join(word_list), 60), sep="\n")

Not found:  1384
一 上 下 世 丟 並 中 久 乎 乘 乾 亂 了 予 事 亡 交 亮 人 付 令 以 仰 伏 伙 估 伸 伺 佇 佈
位 低 住 佔 作 使 來 依 侵 便 修 俯 俱 倒 借 做 停 偵 偶 備 催 傳 債 傷 傻 僱 價 僻 儲 充
光 克 兒 入 全 共 兵 具 兼 冶 凍 凶 出 函 分 刊 划 列 別 刪 到 制 刷 刻 剖 剪 割 創 劃 劈
加 勒 動 勝 勳 勻 包 化 匯 升 卡 危 厭 厲 去 參 及 取 受 口 叨 叩 叫 史 合 吊 同 吐 向 吞
呆 呈 味 命 和 咬 哄 哭 售 唯 問 啞 善 喊 喘 喙 喳 嘆 嚐 嚴 嚼 回 囤 固 國 圓 土 坐 垂 垮
埋 堅 報 場 堵 塌 塑 塞 填 墜 增 墾 壓 壞 壯 多 夠 大 天 失 夾 奉 奏 奔 奮 好 妄 妥 婷 嫁
嬰 孕 存 守 完 定 害 容 寂 寐 寒 察 寡 實 審 寫 封 射 尊 尋 對 尚 就 居 屈 屋 展 山 岔 岸
峙 崖 崩 巡 師 帶 平 幾 廠 建 弄 弓 引 弛 張 彌 往 得 徙 從 心 志 忙 念 忽 怖 思 急 性 怨
怯 恃 恥 恨 息 悉 悔 悚 悠 惡 想 愛 慢 慣 慰 慾 憑 憤 憨 懂 懷 成 戒 截 戴 扁 扎 打 托 扛
扳 找 抄 抉 抓 抗 折 披 抱 抵 抹 押 拋 拍 拐 拓 拗 拙 招 拜 拳 拷 挖 挪 挫 挽 挾 捅 捆 捕
捱 捲 掀 掃 授 掉 掏 掘 掛 掠 採 探 接 控 掩 換 揮 援 揹 損 搜 搞 搬 搭 摔 摩 摹 撒 撞 撥
撫 播 撰 撲 撼 撿 擇 擊 操 擠 擦 擲 擴 擾 攝 攢 攪 收 攻 放 救 敘 教 散 敲 敵 斃 文 斜 斥
新 斷 於 施 日 早 昂 明 昏 映 晉 普 曬 曳 會 有 服 朗 望 未 本 枝 架 染 柔 查 栓 校 核 根
格 栽 桿 梭 梳 棄 棒 棲 植 楞 榨 樹 機 檄 檔 檢 櫃 權 欣 款 歇 歛 歡 止 正 步 歸 死 殷 毀
比 民 氣 水 求 決 沈 沉 沒 沖 治 沿 泊 法 泛 泡 泣 泥 洋 洌 洒 洗 洩 活 派 流 涉 液 涼 淌
淘 淚 淡 淪 深 混 添 清 減 測 湧 準 溜 溯 溶 滅 滲 滴 滾 滿 漏 演 漱 漲 潤 澹 激 濟 濾 瀉
灑 火 灰 灼 為 烈 烘 烤 無 然 煉 煞

## Words cannot be found in tencent

In [15]:
word_list = list(word_set-set(vocabs))
word_list = sorted(word_list, key=lambda x: (len(x),x))
print("Found: ", len(word_list))
print(*textwrap.wrap(" ".join(word_list), 60), sep="\n")

Found:  1177
攙 癟 訃 躓 輾 醞 隄 丟來 丟光 丟出 丟到 丟命 丟擲 丟盡 丟給 丟荒 並比 並置 並陳 交到 交匯 交屋
交援 交溶 交磕 交納 交絡 交纏 交纓 交著 交詮 交踏 交辦 交迸 交逼 交關 交馳 交齊 仰戴 仰觀 伸回 伸引
伸進 伸過 佈上 佈建 佈放 佈新 佈有 佈設 侵厲 修來 修到 修滿 修畢 修竣 修編 修茂 修蓋 修觀 俱長 停佇
停借 停到 停定 停慢 停棲 停發 停租 停筆 停腳 停開 停附 催動 催繳 催趕 催踢 催迷 僻居 充啞 充為 充脹
充食 克奏 具帶 具現 具瞻 兼包 兼愛 兼攻 兼負 兼賣 兼顧 冶性 冶熔 凍上 凍成 凍掉 凍斃 凍紅 凍餓 凍黏
刊於 刊有 刊證 刪掉 刪減 剖空 剖陳 剪作 剪貼 割成 割給 劃上 劃下 劃亮 劃傷 劃動 劃為 劃破 劃給 劃過
劃銷 劈來 劈去 勒定 勒戒 勻勻 勻婷 勻美 厭勝 厭恨 厭氣 厭身 去國 去寒 去枝 去鄉 受僱 受勳 受報 受完
受敵 受有 受灼 受盡 受領 叨回 叫來 叫修 叫入 叫到 叫慣 叫為 叫煩 叫窮 叫貨 叫進 叫響 吞進 呆藏 哄亂
哄累 售屋 售給 售與 喊到 喊去 喊為 喘氣 嚐下 嚐出 嚐盡 嚐足 嚼出 嚼動 囤壓 囤積 固接 固著 埋去 埋進
埋頭 堅留 堅辭 堅邀 堵回 堵車 填到 填成 填緊 填進 填鋪 填飽 填鴨 墜上 墜崖 壞掉 壞損 壞滅 夾上 夾傷
夾出 夾到 夾峙 夾斷 夾有 夾痛 夾織 夾蔭 夾處 夾起 夾進 奉伺 奉成 奉有 奉核 奉納 奉請 孕生 容共 寂止
寡佔 寡歡 寡聞 審思 審校 審酌 審量 屈伏 屈守 屈沈 屈體 崩倒 崩飛 帶位 帶傷 帶到 帶大 帶好 帶引 帶怯
帶慣 帶成 帶淚 帶滿 帶班 帶累 帶給 帶血 帶行 帶讀 帶起 帶返 帶開 帶隊 帶離 幾經 幾諫 幾近 張口 張喙
張弛 張結 張致 張閤 彌平 從便 從眾 忙到 思問 思算 恥於 恥言 悔斷 悔醒 悠到 悠邈 悠鳴 想盡 慣看 慣稱
憑依 憑吊 憑添 憑著 憨呆 憨拙 懷有 懷柔 懷藏 扎來 扎完 打上 打封 打直 打紅 托來 托兒 托嬰 托給 找來
找對 找沒 找無 找給 找錯 抄下 抄出 抄封 抄有 抓來 抓傷 抓勻 抓權 抓準 抓痛 抓硬 抓穩 抓舉 抓蝦 抓進
抓開 抓食 折拗 折換 折現 折轉 披瀉 披蓋 抱來 抱

## Output Hashes

```
../data/verb_morpho.csv 4ecd72
../data/verb_morpho_emb.txt a9a9b8
```

In [16]:
from hashlib import sha1
from pathlib import Path
for path_x in output_paths:
    h = sha1()
    h.update(Path(path_x).read_bytes())
    print(path_x, h.hexdigest()[:6])

../data/verb_morpho.csv 4ecd72
../data/verb_morpho_emb.txt a9a9b8
